## Goal 
 - Manually update the CSVs of a few stocks
 - Write a function to update a given CSV with that tickers data

----------------

### TODO
 - Think of scenarios where this might fail, and what I can reasonably do to combat this
 - Write and schedule procedure to scrape for all **stock_data** stocks Monday-Friday after close
 - Write and schedule procedure to scrape for all S&P 500 stocks Monday-Friday after close

In [1]:
from stock_utils import *

In [2]:
from bs4 import BeautifulSoup
import requests
import re

### Yahoo

In [6]:
def update_CSV(path_to_csv, backup=False):
    if backup is True:
        raise ValueError('Not yet supported')
        
    ticker = ticker_from_csv(path_to_csv)
        
    ## Open Yahoo Finance data
    url = "https://finance.yahoo.com/quote/{sym}/history?p={sym}".format(sym=ticker)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    ## Store all rows of historical data for this stock in 'all_rows'
    all_rows = []
    historical_rows = soup.findAll('tr', attrs={'class':'BdT Bdc($c-fuji-grey-c) Ta(end) Fz(s) Whs(nw)'})
    for row in historical_rows:
        table_cells = row.findAll('td')
        row_list = []
        for cell in table_cells:
            #print(cell.text.strip())
            row_list.append(cell.text.strip())
        all_rows.append(row_list)
        
    ## Add any days that are missing to the CSV
    df = pd.DataFrame()
    df = df.from_csv(path_to_csv)
    
    for i in range(len(all_rows)):
        if pd.to_datetime(all_rows[i][0]) not in df.index:
            s_close = all_rows[i][5]
            s_volume = int(all_rows[i][6].replace(',', ''))
            s_open = all_rows[i][1]
            s_high = all_rows[i][2]
            s_low = all_rows[i][3]
            df.loc[pd.to_datetime(all_rows[i][0])] = [s_close, s_volume, s_open, s_high, s_low]
      
    df = df.sort_index(axis=0)
    df.to_csv(path_to_csv)

In [17]:
def update_csv(arg):
    update_CSV(arg)

In [7]:
#update_CSV('stock_data/amd.csv')

In [16]:
g = glob.glob('stock_data/*.csv')
for file in g:
    update_CSV(file)